In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d kazanova/sentiment140

from zipfile import ZipFile

dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("The Dataset is extracted")

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')

print(stopwords.words('english'))

df=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

df.shape

df.head()

column_names=['target','id','date','flag','user','text']
df=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')

df.head()

df.isnull().sum()

df['target'].value_counts()

df.replace({'target':{4:1}}, inplace=True)

df['target'].value_counts()

port_stem=PorterStemmer()

def stemming(content):

  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)

  return stemmed_content

df['stemmed_content']=df['text'].apply(stemming)

df.head()

x=df['stemmed_content'].values
y=df['target'].values

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

vectorizer=TfidfVectorizer()
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

lr=LogisticRegression(max_iter=1000)

lr.fit(x_train,y_train)

y_pred=lr.predict(x_test)

acc=accuracy_score(y_test,y_pred)
print(acc)





import pickle

filename='trained_model.sav'
pickle.dump(lr,open(filename,'wb'))

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

x_new=x[6000]
print(y[6000])
pred=loaded_model.predict(x_new)
print(pred)

if(pred[0]==0):
  print("Negative")
else:
  print("Positive")